In [46]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
# get data
data = pd.read_csv('local_guide.csv')

# create object TfidfVectorizer
vectorizer = TfidfVectorizer()

# Melakukan vektorisasi TF-IDF pada fitur "Tempat"
tfidf_matrix_tempat = vectorizer.fit_transform(data['Tempat'])

# Mengubah matriks TF-IDF menjadi array
tfidf_matrix = tfidf_matrix_tempat.toarray()

# loads model
model = load_model('model_local_guide.h5')

In [48]:
# Merekomendasikan item berdasarkan Tempat
def recommend_items(tempat, tfidf_matrix, model, items=data[['Pemandu_ID', 'Nama_Pemandu', 'Optional_Bahasa', 'Umur', 'Jenis_Kelamin', 'Tempat', 'Pendidikan_Terakhir', 'Pekerjaan', 'Nomor_Telepon', 'Price_per_hour', 'Time_duration_in_min', 'Avatars', 'Rating']], k=5):
    # Mendapatkan indeks item berdasarkan input Tempat
    def get_item_index_by_tempat(tempat, data):
        index = data[data['Tempat'] == tempat].index
        return index[0] if len(index) > 0 else None

    indeks_item = get_item_index_by_tempat(tempat, data)
    if indeks_item is None:
        return pd.DataFrame()  # Mengembalikan dataframe kosong jika Tempat tidak ditemukan

    item_embedding = tfidf_matrix[indeks_item]
    item_embedding = np.reshape(item_embedding, (1, -1))

    similarity_scores = cosine_similarity(tfidf_matrix, item_embedding)
    similarity_scores = similarity_scores.flatten()

    indeks_terurut = np.argsort(similarity_scores)[::-1][:k]
    item_terrekomendasikan = items.iloc[indeks_terurut].copy()  # Add .copy() here

    # Mengurutkan berdasarkan Tempat terbaik
    item_terrekomendasikan = item_terrekomendasikan.sort_values('Tempat', ascending=False)

    # Prediksi menggunakan model
    item_embedding_pred = model.predict(item_embedding)
    similarity_scores_pred = cosine_similarity(tfidf_matrix, item_embedding_pred)
    similarity_scores_pred = similarity_scores_pred.flatten()

    indeks_terurut_pred = np.argsort(similarity_scores_pred)[::-1][:k]
    item_terrekomendasikan_pred = items.iloc[indeks_terurut_pred].copy()  # Add .copy() here

    # Mengurutkan berdasarkan Tempat terbaik
    item_terrekomendasikan_pred = item_terrekomendasikan_pred.sort_values('Tempat', ascending=False)

    return item_terrekomendasikan_pred

In [49]:
# run program
tempat_input = 'Bali'  # Ganti dengan input Tempat yang diinginkan

item_terrekomendasikan = recommend_items(tempat_input, tfidf_matrix, model)


1/1 [==============================] - 0s 423ms/step


In [50]:
# show result
if item_terrekomendasikan.empty:
    print('Tidak ada item yang ditemukan untuk Tempat yang diberikan.')
else:
    # Menampilkan item yang direkomendasikan
    print(item_terrekomendasikan[['Pemandu_ID', 'Nama_Pemandu', 'Optional_Bahasa', 'Umur', 'Jenis_Kelamin', 'Tempat', 'Pendidikan_Terakhir', 'Pekerjaan', 'Nomor_Telepon', 'Price_per_hour', 'Time_duration_in_min', 'Avatars', 'Rating']])

     Pemandu_ID                   Nama_Pemandu Optional_Bahasa  Umur  \
1916     PMD916  drg. Gandi Nashiruddin, S.Ked         Inggris    31   
1130    PMD1936                   Jais Mustofa        Mandarin    30   
1538     PMD538                Umaya Sihombing          Jepang    20   
1131    PMD1937     H. Muhammad Prabowo, S.E.I         Inggris    29   
683      PMD153                  Endra Siregar         Inggris    30   

     Jenis_Kelamin Tempat Pendidikan_Terakhir  Pekerjaan  Nomor_Telepon  \
1916     Perempuan   Bali                  S2   Wartawan  6288955369534   
1130     Laki-laki   Bali                 SMA  Pengusaha  6288544167597   
1538     Laki-laki   Bali                 SMP   Wartawan  6281334498718   
1131     Laki-laki   Bali                  S1  Pengusaha  6286929231916   
683      Perempuan   Bali                  S2        PNS  6288341841934   

      Price_per_hour  Time_duration_in_min  \
1916           98000                   360   
1130          108000    

In [51]:
# show as dataframe
item_terrekomendasikan

,Pemandu_ID,Nama_Pemandu,Optional_Bahasa,Umur,Jenis_Kelamin,Tempat,Pendidikan_Terakhir,Pekerjaan,Nomor_Telepon,Price_per_hour,Time_duration_in_min,Avatars,Rating
1916,PMD916,"drg. Gandi Nashiruddin, S.Ked",Inggris,31,Perempuan,Bali,S2,Wartawan,6288955369534,98000,360,xsgames.co/randomusers/avatar.php?g=female,2
1130,PMD1936,Jais Mustofa,Mandarin,30,Laki-laki,Bali,SMA,Pengusaha,6288544167597,108000,180,xsgames.co/randomusers/avatar.php?g=male,1
1538,PMD538,Umaya Sihombing,Jepang,20,Laki-laki,Bali,SMP,Wartawan,6281334498718,75000,120,xsgames.co/randomusers/avatar.php?g=male,4
1131,PMD1937,"H. Muhammad Prabowo, S.E.I",Inggris,29,Laki-laki,Bali,S1,Pengusaha,6286929231916,137000,60,xsgames.co/randomusers/avatar.php?g=male,1
683,PMD153,Endra Siregar,Inggris,30,Perempuan,Bali,S2,PNS,6288341841934,58000,300,xsgames.co/randomusers/avatar.php?g=female,2
